In [2]:
import psycopg2
import folium
import pandas as pd
import ast 
from shapely.geometry import Point
import shapely
from math import inf
import geopy.distance as gp

In [64]:
connection = psycopg2.connect(user="postgres",
                                password="postgres",
                                host="127.0.0.1",
                                port="5432",
                                database="map")                                
cursor = connection.cursor()

In [65]:
cursor.execute("select coordinates from geolocation full outer join object on geolocation.geo_id=object.geo_id where object.type = 1")
record = cursor.fetchall()

In [66]:
object_coordinates = []
for i in range(0, len(record)):
    buff = record[i][0].replace('(', '').replace(')', '')
    buff = [float(val) for val in buff.split(',')]
    object_coordinates.append(buff[::-1]) 

In [3]:
main_table = pd.read_csv('data/main_table.csv', sep=';')
main_table.head(3)

,zid,adm_zid,reverse_coordinates,customers_cnt_home_in_zone,customers_cnt_home_in_adm_zone,contains_object_1,contains_object_2,contains_object_4,contains_object_5,contains_object_6,contains_object_7,part_of_customer_cnt_home_in_adm_zone
0,32909,216,"[[[55.144778403557694, 37.03345981389939], [55...",4,18099,0,0,0,0,0,0,0.022101
1,32910,216,"[[[55.14031640325421, 37.041353646223094], [55...",25,18099,0,0,0,0,0,0,0.138129
2,33261,216,"[[[55.149239886020204, 37.02556422002536], [55...",2,18099,0,0,0,0,0,0,0.011050


In [163]:
def create_recommendation_polygons(object_Type):

    polygons_length = 600

    cursor.execute(f"select coordinates from geolocation full outer join object on geolocation.geo_id=object.geo_id where object.type = {object_Type}")
    record = cursor.fetchall()

    object_coordinates = []
    for i in range(0, len(record)):
        buff = record[i][0].replace('(', '').replace(')', '')
        buff = [float(val) for val in buff.split(',')]
        object_coordinates.append(buff[::-1])

    main_table = pd.read_csv('data/main_table.csv', sep=';')

    buff = main_table[['reverse_coordinates', 'part_of_customer_cnt_home_in_adm_zone', f'contains_object_{object_Type}']].sort_values(by=['part_of_customer_cnt_home_in_adm_zone', f'contains_object_{object_Type}' ], ascending=False, ignore_index=True)

    polygons = []

    i = 0
    while len(polygons) < polygons_length:
        if buff['part_of_customer_cnt_home_in_adm_zone'][i] / buff[f'contains_object_{object_Type}'][i] == inf:
                reverse_coordinates = ast.literal_eval(buff['reverse_coordinates'][i])
                count = 0 
                for object in object_coordinates:
                        if gp.geodesic((object), (reverse_coordinates[0][0])).km > 1.5:
                                count +=1
                if count == len(object_coordinates):
                        polygons.append(reverse_coordinates)
        i += 1

    return object_coordinates, polygons

    
object_coordinates, polygons  = create_recommendation_polygons(4)
    

/Users/tim/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in double_scalars


In [127]:
recommendation_polygons = pd.DataFrame()

In [169]:
recommendation_polygons['4_type_object_recommendation_polygons'] = polygons

In [170]:
recommendation_polygons

,2_type_object_recomendation_polygons,1_type_object_recomendation_polygons,5_type_object_recomendation_polygons,6_type_object_recomendation_polygons,7_type_object_recomendation_polygons,4_type_object_recomendation_polygons
0,"[[[55.94507886554302, 37.34402777340144], [55....","[[[55.98068138176378, 37.17565563034755], [55....","[[[55.98068138176378, 37.17565563034755], [55....","[[[55.846344403032, 37.57593689611744], [55.84...","[[[55.84634907015495, 37.56795368088328], [55....","[[[55.482388652529465, 37.298403431852286], [5..."
1,"[[[55.94058756637614, 37.34404582160569], [55....","[[[55.82388226205482, 37.583871764065705], [55...","[[[55.98068138176378, 37.17565563034755], [55....","[[[55.7430463510216, 37.567774053758654], [55....","[[[55.84185768500604, 37.56794584675406], [55....","[[[55.8142940222839, 37.894942056606226], [55...."
2,"[[[55.90919245984965, 37.384152527043376], [55...","[[[55.98068138176378, 37.17565563034755], [55....","[[[55.81888364387522, 37.86307788584394], [55....","[[[55.66221757288794, 37.51201437706986], [55....","[[[55.8418530186659, 37.57592814163483], [55.8...","[[[55.487031837103196, 37.55942349197907], [55..."
3,"[[[55.59933339407349, 37.47233019209342], [55....","[[[55.81888364387522, 37.86307788584394], [55....","[[[55.98070219309872, 37.183666380494564], [55...","[[[55.895744232373026, 37.58402671842871], [55...","[[[55.94507886554302, 37.34402777340144], [55....","[[[55.599331347750386, 37.46439720187345], [55..."
4,"[[[55.60342796321443, 37.18670928458309], [55....","[[[55.82387656396234, 37.59185037784273], [55....","[[[55.97621097190184, 37.18370303367466], [55....","[[[55.62628274646946, 37.472311209861104], [55...","[[[55.94058756637614, 37.34404582160569], [55....","[[[55.58542041205245, 37.17099225723423], [55...."
...,...,...,...,...,...,...
595,"[[[56.01200781936465, 37.13530615805209], [56....","[[[55.82391026005317, 37.51206415776563], [55....","[[[55.88223643870304, 37.62395049657294], [55....","[[[55.747502171309684, 37.61555997441973], [55...","[[[55.725099098160285, 37.49611673616849], [55...","[[[55.44645552753642, 37.29858666688487], [55...."
596,"[[[55.51399553902578, 37.48822172758896], [55....","[[[55.747212580322106, 37.79074487842871], [55...","[[[55.62628528371327, 37.51200338336169], [55....","[[[55.846344403032, 37.57593689611744], [55.84...","[[[55.60824094063379, 37.638954561740064], [55...","[[[55.464380780381525, 37.2747776212768], [55...."
597,"[[[55.571809054557555, 37.12354106768256], [55...","[[[55.89574942695113, 37.576033362585754], [55...","[[[55.82840218686567, 37.504086002952725], [55...","[[[55.50938640456526, 37.32992512522109], [55....","[[[55.69813940125534, 37.44840146301595], [55....","[[[55.44946004220119, 36.863907699438066], [55..."
598,"[[[55.5763712768339, 37.1472826824369], [55.57...","[[[55.86866902062195, 37.69579760263061], [55....","[[[55.63972645154641, 37.591419034735665], [55...","[[[55.70675117135187, 37.80635393847342], [55....","[[[55.56330654994592, 37.34554375462652], [55....","[[[55.4416691233949, 37.16428778171801], [55.4..."


In [171]:
recommendation_polygons.to_csv('data/recommendation_polygons.csv', sep=';', index=False)